<div style="background-color: #f9f9f9; padding: 10px; border-radius: 5px; color: #333;">
  
# 🏆 **Scraping the _IMDB_ 🎬 Top Productions Companies**  

</div>

### 📦 **Libraries Used for Scraping _IMDB_ 🎬**  

In [2]:
import pandas as pd 
import numpy as np
import requests
from bs4 import BeautifulSoup
from scrapy import Selector

##  📦 Extracting Data with **BeautifulSoup** 

In [ ]:
base_url = "https://www.imdb.com"

start_url ="https://www.imdb.com/list/ls567506715/"

headers = ({'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36'})

response = requests.get(start_url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

company_links = soup.find_all("a", href=lambda href: href and href.startswith("/company/co"))[:20]

sorted_links = []

for link in company_links:
    relative_url = link["href"]  
    full_url = base_url + relative_url 

    redirected_response = requests.get(full_url, headers=headers, allow_redirects=True)
    final_url = redirected_response.url  

    sorted_url = final_url + "&sort=boxoffice_gross_us,desc"
    sorted_links.append(sorted_url)

In [ ]:
len(sorted_links)

20

## 📦 Extracting the **Production company Data links**

In [ ]:
movie_links=[]
for links in sorted_links:
    response2=requests.get(links,headers=headers)
    select=Selector(text=response2.text)
    link=select.css('.ipc-title-link-wrapper::attr(href)').getall()[:10]
    movie_links.append(link)

## 📦 Extracting the **Production company Details**

In [ ]:
movie_name=[]
release_year =[]
certificate = []
duration=[]
rating=[]
director=[]
oscar_nominations=[]
genre=[]
countries=[]
language=[]
production_company=[]
budget=[]
gross_worldwide=[]

for i in movie_links:
    for link in i:
        links="https://www.imdb.com"+link
        response3=requests.get(links,headers=headers)
        sle=Selector(text=response3.text)

        n=sle.css('div.sc-9a2a0028-3.bwWOiy > div.sc-70a366cc-0.bxYZmb > h1 > span::text').get()
        movie_name.append(n)

        release = sle.css("div.sc-9a2a0028-3.bwWOiy > div.sc-70a366cc-0.bxYZmb > ul > li:nth-child(1) > a::text").get()
        release_year.append(release)

        certi = sle.css("div.sc-9a2a0028-3.bwWOiy > div.sc-70a366cc-0.bxYZmb > ul > li:nth-child(2) > a::text").get()
        certificate.append(certi)

        durat = sle.css("div.sc-70a366cc-0.bxYZmb > ul > li:nth-child(3)::text").get()
        duration.append(durat)

        ratin= sle.css('.sc-d541859f-1.imUuxf::text').get()
        rating.append(ratin)

        direct = sle.css("div.sc-70a366cc-2.bscNnP > div > ul > li:nth-child(1) > div > ul > li > a::text").get()
        director.append(direct)

        oscar_nomi = sle.css("div > ul > li > a.ipc-metadata-list-item__label.ipc-metadata-list-item__label--link::text").getall()
        if len(oscar_nomi) > 2:
                splited_oscar=oscar_nomi[2].split()
                if len(splited_oscar) >2 and splited_oscar[2].isdigit():
                       oscar_nomination =int(splited_oscar[2])
                else: 
                   oscar_nomination = 0
        else :
              oscar_nomination = 0
        oscar_nominations.append(oscar_nomination)

        genr = sle.css("div.ipc-chip-list__scroller > a > span::text").getall()
        genre.append(genr)

        country = sle.css(
                  "div.sc-f65f65be-0.dQVJPm > ul > li:nth-child(2) > div > ul > li:nth-child(1) > a::text, \
                    div.sc-f65f65be-0.dQVJPm > ul > li:nth-child(2) > div > ul > li:nth-child(2) > a::text").getall()
        countrie = country[:2]
        countries.append(countrie)

        lang = sle.css("div.sc-f65f65be-0.dQVJPm > ul > li:nth-child(4) > div > ul > li > a::text").getall()
        language.append(lang)

        production = sle.css("div.sc-f65f65be-0.dQVJPm > ul > li:nth-child(7) > div > ul > li> a::text").getall()
        production_company.append(production)

        bud = sle.css("div.sc-f65f65be-0.dQVJPm > ul > li:nth-child(1) > div > ul > li > span::text").get()
        budget.append(bud)

        gross =sle.css("div.sc-f65f65be-0.dQVJPm > ul > li:nth-child(4) > div > ul > li > span::text").get()
        gross_worldwide.append(gross)


## 📊 Converting Data to a **Pandas DataFrame**  


In [ ]:
df = pd.DataFrame({'movie_name':movie_name,'release_year':release_year,'certificate':certificate,'duration':duration,'rating':rating,'director':director,
    'oscar_nominations':oscar_nominations,
    'genre':genre,'countries':countries,'language':language,'production_company':production_company,'budget':budget,'gross_worldwide':gross_worldwide
})
df.head()

,movie_name,release_year,certificate,duration,rating,director,oscar_nominations,genre,countries,language,production_company,budget,gross_worldwide
0,The Polar Express,2004,U,1h 40m,6.6,Robert Zemeckis,3,"[Computer Animation, Holiday Animation, Holida...","[United States, Australia]",[English],"[Castle Rock Entertainment, Shangri-La Enterta...","$165,000,000 (estimated)","$318,910,211"
1,A Few Good Men,1992,U,2h 18m,7.7,Rob Reiner,0,"[Legal Drama, Legal Thriller, Drama, Thriller]","[United States, Color]",[],[],"$41,000,000 (estimated)",Cuestión de honor
2,The Green Mile,1999,UA,3h 9m,8.6,Frank Darabont,4,"[Period Drama, Prison Drama, Supernatural Fant...","[United States, Color]","[English, French]","[Castle Rock Entertainment, Darkwoods Producti...","$60,000,000 (estimated)","$286,801,374"
3,City Slickers,1991,PG-13,1h 53m,6.8,Ron Underwood,0,"[Buddy Comedy, Contemporary Western, High-Conc...","[United States, Color]",[],[],"$27,000,000 (estimated)",Cowboys de ciudad
4,Miss Congeniality,2000,A,1h 49m,6.3,Donald Petrie,0,"[High-Concept Comedy, Action, Comedy, Crime, R...","[United States, Color]","[English, Russian, Hawaiian]","[Castle Rock Entertainment, Village Roadshow P...","$45,000,000 (estimated)","$212,742,720"


In [ ]:
df.shape

(190, 13)

## 📝 Saving Scraped **IMDB Data to CSV**  

In [ ]:
df.to_csv('../data/raw/data_scraping_imdb.csv')